In [1]:
from PIL import Image, ImageDraw
from face_recognition import load_image_file, face_landmarks

In [2]:
img_array = load_image_file("images/thispersondoesnotexist.jpg")
landmarks = face_landmarks(img_array,  model="small")

In [3]:
img = Image.fromarray(img_array)
draw = ImageDraw.Draw(img)
mustache = Image.open("images/mustache.png")

In [4]:
def x_small(coordinates):
    return min(coordinates)[0]
    
def x_large(coordinates):
    return max(coordinates)[0]

In [5]:
from math import ceil

for face in landmarks:
    eyeWidth = x_large(face["right_eye"]) - x_small(face["left_eye"])
    nosetip = face["nose_tip"][0]

    x, y = mustache.size
    ratio = eyeWidth / x * 1.5
    new_x, new_y = x*ratio, y*ratio
    mask = mustache.resize((ceil(new_x), ceil(new_y)))
    pos = nosetip[0]-round(new_x/2), nosetip[1] - round(new_y/2.4)
    img.paste(mask, pos, mask)

In [6]:
img.show()

In [7]:
def triangle(landmarks):
    """
    Find 3 corner points to transform mustache
    :param landmarks: all landmarks
    :return: triple of outer eyes and nose tip
    """
    a = min(landmarks["left_eye"])
    b = max(landmarks["right_eye"])
    c = landmarks["nose_tip"][0]
    return [a, b, c]






In [8]:
import cv2 as cv
from cv2 import getAffineTransform, warpAffine, imread, samples, imshow
import numpy as np

In [9]:
# src = cv.imread("images/thispersondoesnotexist.jpg")
# if src is None:
#     print('Could not open or find the image:', args.input)
#     exit(0)


def transform(mustache_img, dstTri):
    src = np.array(mustache_img)
    src = cv.imread("images/thispersondoesnotexist.jpg")

    srcTri = [[0,0], [src.shape[0], 0], [src.shape[0]/2, src.shape[1]]]

    dstTri = np.array(dstTri).astype(np.float32)
    srcTri = np.array(srcTri).astype(np.float32)

    warp_mat = getAffineTransform(srcTri, dstTri)
    warp_dst = warpAffine(src, warp_mat, (src.shape[1], src.shape[0]))
    return warp_dst

transformed_img = transform(mustache, triangle(landmarks[0]))

In [10]:
    # cv.imshow("transformed img", transformed_img)
    # cv.waitKey(0)

In [11]:
def cv2pil(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return Image.fromarray(img)

mus = cv2pil(transformed_img)

In [12]:
mus.show()